<a href="https://colab.research.google.com/github/SUYAMBULAKSHMI-VENKATESAN/NAAN-MUDHALVAN-AI/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pickle
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

with open("dialogs.txt", 'r', encoding='utf-8') as file:
    data = file.read()
print(data)


# Tokenization
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
sentences = data.split('\n')


print(sentences)

# Preprocessing
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Preprocess data
preprocessed_sentences = [preprocess_text(sentence) for sentence in sentences]
print(preprocessed_sentences)


input_texts = []
target_texts = []

lines = preprocessed_sentences
for i in range(0, len(lines) - 1, 2):
    input_texts.append(lines[i].strip())  # User input (input text)
    target_texts.append(lines[i + 1].strip())


print(input_texts)
print(target_texts)

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

# Convert input texts and target texts into numerical form
X_input = vectorizer.fit_transform(input_texts)
X_target = vectorizer.fit_transform(target_texts)

print(X_input)
print(X_target)

from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

# Tokenizing input and target texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_texts + target_texts)

input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)

max_sequence_length = max(len(seq) for seq in input_sequences)  # Adjust if input and target lengths differ

# Padding sequences to make them uniform in length
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length)
padded_target_sequences = pad_sequences(target_sequences, maxlen=max_sequence_length)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 for 0 padding

# Building the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=max_sequence_length))
model.add(Masking(mask_value=0))  # Masking zero-padding
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model
model.fit(padded_input_sequences, np.expand_dims(padded_target_sequences, axis=-1), epochs=50, batch_size=32)

# Save the trained model
model.save('chatbot_lstm_model.h5')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import random



# Preprocess your data if necessary
# Initialize CountVectorizer to convert text data to vectors
vectorizer = CountVectorizer()

# Convert input texts and target texts into numerical form
X_input = vectorizer.fit_transform(input_texts)
y = target_texts


# Train a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_input, y)

# Example user input
user_query = input()

# Convert the user input into vector form
user_query_vector = vectorizer.transform([user_query])

# Predict the response using the trained classifier
predicted_response = clf.predict(user_query_vector)

print("User Query:", user_query)
print("Bot Response:", predicted_response[0])
